In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import camb 
from camb import model
print(f'Using CAMB {camb.__version__} installed at {os.path.dirname(camb.__file__)}')

Using CAMB 1.5.8 installed at /home/gilee/CAMB/camb


In [9]:
def initialize_camb_params(cosmo_params, z):

    h = cosmo_params['h']
    Omega_b = cosmo_params['Omega_b']
    Omega_c = cosmo_params['Omega_c']
    Omega_k = cosmo_params['Omega_k']
    w0 = cosmo_params['w0']
    wa = cosmo_params['wa']
    As = cosmo_params['As']
    ns = cosmo_params['ns']
    kmax = cosmo_params['kmax']
    k_per_logint = cosmo_params['k_per_logint']
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h, ombh2=Omega_b*h*h, omch2=Omega_c*h*h, omk=Omega_k)
    pars.DarkEnergy = camb.dark_energy.DarkEnergyPPF(w=w0, wa=wa)
    pars.InitPower.set_params(As=As, ns=ns)
    pars.set_matter_power(redshifts=z, kmax=kmax, k_per_logint=k_per_logint)
    pars.NonLinear = model.NonLinear_none
    
    return pars

In [10]:
def adjust_As_at_z0(pars, cosmo_params):
    
    z0 = pars.Transfer.PK_redshifts[-1] # as CAMB sorts redshifts (earliest first)

    if z0 != 0:
        print(f"Redshifts should incldue 0 here, as we normalize sigma8 at z=0.")
        return None

    results = camb.get_results(pars)
    sigma8_calc = results.get_sigma8()[-1]
    sigma8_goal = cosmo_params['sigma8_z0_WMAP5']

    print(f"Target     sigma8 = {sigma8_goal} at z = {z0}")
    print(f"Initial    sigma8 = {sigma8_calc}, As = {pars.InitPower.As} (trial)")

    As = pars.InitPower.As * (sigma8_goal / sigma8_calc)**2 # new normalization
    pars.InitPower.set_params(As=As, ns=pars.InitPower.ns) # set new parameterization

    # Get new results and sigma8 value
    results = camb.get_results(pars)
    sigma8 = results.get_sigma8()[-1]
    print(f"Normalized sigma8 = {sigma8}, diff={sigma8 - sigma8_goal}, As = {pars.InitPower.As} (new)")
    
    return pars

In [11]:
bias8_z0_WMAP5  = 1.26             # target value # bias8 at z=0
sigma8_z0_WMAP5 = 1/bias8_z0_WMAP5 # target value # sigma8 at z=0

As0       = 2e-9                   # trial value # power spectrum noramlization factor at k=k0
ns        = 0.96                   # power spectral index
h         = 0.72                   # little h, dimensionless hubble parameter, H0 = 100*h [km/s/Mpc]

Omega_m   = 0.26                   # physical total matter density
Omega_b   = 0.044                  # physical baryon density
Omega_c   = Omega_m - Omega_b      # physical CDM (cold dark mattery) density
Omega_l   = 1 - Omega_m            # DE (dark energy) density
Omega_k   = 0.0                    # spatial curvature

# EoS of DE
w0 = -1
wa = 0

In [12]:
# starting redshift
z_start = 200
a_start = 1/(z_start+1)

# simulation box setups (See Jupyter Notebook 00. for details)
levelmin = 10 # initial level
levelmax = levelmin + 7 # final level at z=0
# Here, `7` is fixed number, as this is the number of global refinement in the ramses code.
n_cell0  = 2**levelmin # initial cell numbers
c_Lbox   = 2**levelmax / 1000 # [cMpc] # comoving box size
c_dx0    = c_Lbox / n_cell0   # [cMpc] # comoving initial cell size

print("\nSimulation Setups:")
print("  z_start       = ", z_start)
print("  a_start       = ", a_start)
print("  Lbox [cMpc]   = ", c_Lbox)
print("  Lbox [cMpc/h] = ", c_Lbox*h)
print("  dx_ini [ckpc] = ", c_dx0 * 1000)
print("  dx_ini [pkpc] = ", c_dx0 * 1000 / (z_start + 1))
print("  dx_fin [pkpc] = ", c_dx0 * 1000 / 2**7)
print("  levelmin      = ", levelmin)
print("  levelmax      = ", levelmax)
print("  Ncell_ini     = ", n_cell0)

# power spectrum params
kmin = 2 * np.pi / c_Lbox       # [1/Mpc] # largest scale (box size)
kmax = np.pi * n_cell0 / c_Lbox # [1/Mpc] # smallest scale (Nyquist wavenumber)
khmin = kmin/h                  # [1/Mpc] # largest scale (box size)
khmax = kmax/h                  # [1/Mpc] # smallest scale (Nyquist wavenumber)
k0  = 0.05                      # [1/Mpc] pivot_scalar
kh0 = k0/h                      # [1/Mpc] pivot_scalar
k_per_logint = 0 # default: 0 # affects the number of k values

print("\nPower Sepctrum Parameters:")
print("  kmin   [1/Mpc] = ", kmin)
print("  kmax   [1/Mpc] = ", kmax)


Simulation Setups:
  z_start       =  200
  a_start       =  0.004975124378109453
  Lbox [cMpc]   =  131.072
  Lbox [cMpc/h] =  94.37183999999999
  dx_ini [ckpc] =  128.0
  dx_ini [pkpc] =  0.6368159203980099
  dx_fin [pkpc] =  1.0
  levelmin      =  10
  levelmax      =  17
  Ncell_ini     =  1024

Power Sepctrum Parameters:
  kmin   [1/Mpc] =  0.047936899621426284
  kmax   [1/Mpc] =  24.543692606170257


In [13]:
# Cosmological Parameters
cosmo_params = {
    'As': As0, # trial number (before normalization)
    'ns': ns,
    'h': h,
    'bias8_z0_WMAP5': bias8_z0_WMAP5,
    'Omega_m': Omega_m,
    'Omega_b': Omega_b,
    'Omega_k': Omega_k,
    'w0': w0,
    'wa': wa,
    'kmax': kmax,
    'k_per_logint': k_per_logint
}

cosmo_params['sigma8_z0_WMAP5'] = 1/cosmo_params['bias8_z0_WMAP5']
cosmo_params['Omega_c'] = cosmo_params['Omega_m'] - cosmo_params['Omega_b']
cosmo_params['Omega_l'] = 1 - cosmo_params['Omega_m']

In [14]:
cosmo_params

{'As': 2e-09,
 'ns': 0.96,
 'h': 0.72,
 'bias8_z0_WMAP5': 1.26,
 'Omega_m': 0.26,
 'Omega_b': 0.044,
 'Omega_k': 0.0,
 'w0': -1,
 'wa': 0,
 'kmax': 24.543692606170257,
 'k_per_logint': 0,
 'sigma8_z0_WMAP5': 0.7936507936507936,
 'Omega_c': 0.21600000000000003,
 'Omega_l': 0.74}

In [15]:
z = [z_start, 150, 100, 20, 10, 1, 0] # should contain z_start and 0
z.sort(reverse=True) # should be descending order

In [16]:
pars    = initialize_camb_params(cosmo_params, z=z) # initialize
pars    = adjust_As_at_z0(pars, cosmo_params) # normalization

Target     sigma8 = 0.7936507936507936 at z = 0.0
Initial    sigma8 = 0.7628693474928042, As = 2e-09 (trial)
Normalized sigma8 = 0.7936507936507937, diff=1.1102230246251565e-16, As = 2.1646544338903384e-09 (new)


In [17]:
cosmo_params['As'] = pars.InitPower.As # update the As value

In [23]:
results   = camb.get_results(pars)
sigma8_z0 = results.get_sigma8()[-1]

In [33]:
transfers = results.get_matter_transfer_data()
T = transfers.transfer_data
T.shape # [var, k, z]

(13, 217, 7)

In [41]:
header = "\n".join([f"{key}: {value}" for key, value in cosmo_params.items()])

In [42]:
iz = z.index(z_start)

In [43]:
plus_w0 = '+' if w0 >= 0 else ''
plus_wa = '+' if wa >= 0 else ''
fname = f"camb_transfer_z{z[iz]:03.0f}_w0{plus_w0}{w0:.1f}_wa{plus_wa}{wa:.1f}_klogint{k_per_logint:03d}_lmin{levelmin:02d}.txt"
np.savetxt(f"./{fname}", T[:,:,iz].T, fmt="%.10e", header=header)
print(f"MUSIC input data is created: ./{fname}")

MUSIC input data is created: ./camb_transfer_z200_w0-1.0_wa+0.0_klogint000_lmin10.txt


In [36]:
print("The number of k values = ", T.shape[1])

The number of k values =  217
